In [ ]:
%load_ext autoreload
%autoreload 2
from arpes.all import *
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np
%matplotlib inline

In [ ]:
# Define directory
path = os.path.abspath("G:\My Drive\Box_1_square_renamed")

# Different parameters used to set up scans at SLS
scan_type = '*'
direction = 'X'
phi = '*'
energy = '25eV'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = '*'
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

au_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
au_files = glob.glob(au_full_file)
print(len(au_files))
au_files

In [5]:
au_raw_data_list = {}
for file in au_files:
    #data_list.append(load_data(file, location="ULTRA"))
    data = load_data(file, location="ULTRA")
    # k_data = convert_to_kspace(data.spectrum)
    file_name = file.replace(path, '')[1:-3]
    au_raw_data_list.update({file_name : data})

poly Au used to determine Fermi level. Determine Fermiedge and apply to data below

In [ ]:
au_lt = au_raw_data_list['EDC_X_P95_25eV_UNKN_LT_5eVpass_GoldRef11K_0007']
au_lt.S.plot()

In [ ]:
from arpes.analysis.general import rebin

binned = rebin(au_lt, phi=10).spectrum
binned.plot()

In [ ]:
from arpes.all import AffineBroadenedFD, broadcast_model
fermi_level = binned.sel(phi=slice(-0.2,0.2), eV=slice(-0.2,0.1))
results = broadcast_model(AffineBroadenedFD, fermi_level, "phi")

fig, ax = plt.subplots()
fermi_level.plot(ax=ax)
ax.scatter(*results.F.p("fd_center").G.to_arrays(), color="red")
ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
from arpes.fits.fit_models import LinearModel
lin_mod = LinearModel().guess_fit(results.F.p("fd_center"))
lin_mod.plot()
plt.show()

In [ ]:
edc = au_lt.spectrum
edge = lin_mod.eval(x=edc.phi)

corrected_edc = edc.G.shift_by(edge, "eV")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
corrected_edc.plot(ax=axes[0], vmax=10)
edc.plot(ax=axes[1], vmax=20)
# plot vertical lines for y = 0
for ax in axes:
    ax.set_title("")
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)

In [ ]:
# Define directory
path = os.path.abspath("G:\My Drive\Box_1_square_renamed")

# Different parameters used to set up scans at SLS
scan_type = 'FS'
direction = 'AN'
phi = 'P5'
energy = '*'
polarization = '*'
temp = '*'
e_pass = '*'+'eVpass'
comment = ''
run = '*'.zfill(4)

# Put together string for finding file
parameters = [scan_type, direction, phi, energy, polarization, temp, e_pass, comment, run]
if comment == '':
    parameters.remove(comment)
file_type = 'h5'
file_name = '_'.join(parameters)
file = '.'.join([file_name , file_type])

bi2223_full_file = os.path.join(path, file)
file

In [ ]:
import glob
# Load all files with above specified parameters, * is a placeholder and imports all files in its place
bi2223_files = glob.glob(bi2223_full_file)
print(len(bi2223_files))
bi2223_files

In [13]:
bi2223_data_list = {}
for file in bi2223_files:
    #data_list.append(load_data(file, location="ULTRA"))
    raw_data = load_data(file, location="ULTRA")
    edge = lin_mod.eval(x=raw_data.spectrum.phi)
    corrected_data = raw_data.spectrum.G.shift_by(edge, "eV")
    corrected_data.S.apply_offsets({"beta": -0.0077, "phi": -0.004})
    k_data = convert_to_kspace(corrected_data)
    file_name = file.replace(path, '')[1:-3]
    bi2223_data_list.update({file_name : k_data})

Initalize datasets

In [14]:
lt_an_p5 = bi2223_data_list['FS_AN_P5_25eV_CIRCP_LT_5eVpass_0000']
ht_an_p5 = bi2223_data_list['FS_AN_P5_25eV_CIRCP_HT_10eVpass_0000']


Overview of the LT P5 Dataset

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_an_p5.sel(eV=slice(-0.02,0.02)).sum("eV").plot(ax=ax, cmap='PuBu_r', rasterized=True)

plt.ylim(-0.56, 0.56)
plt.xlim(-0.56, 0.56)
plt.title('')
plt.ylabel('k$_y$ ($\AA^{-1}$)', fontsize=18)
plt.xlabel('k$_x$ ($\AA^{-1}$)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)
# plt.yticks(np.arange(-0.4, 0.5, 0.2))
# plt.xticks(np.arange(-0.4, 0.5, 0.2))

kys = [-0.55, -0.5, -0.45, -0.4, -0.35, -0.30, -0.25, -0.22, -0.2]
kxs = [0.16, 0.18, 0.21, 0.24, 0.27, 0.32, 0.38, 0.44, 0.49]
for kx, ky in zip(kxs, kys):
    plt.scatter(ky, kx, color='red', s=50, marker='x')

ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax, fraction=0.1, pad=0.05, aspect=25)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.ax.text(0.6, 1.0, 'Max', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.ax.text(0.6, -0.06, 'Min', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.set_ticks([])
# set x and y axis to be equal
# plt.gca().set_aspect('equal', adjustable='box')
# plt.savefig('Bi2223_wmarkers.pdf',bbox_inches='tight')
plt.show()

In [32]:
def normalize(data):
    return (data - data.sel(eV=slice(0.05,0.07)).mean("eV")) / data.sel(eV=slice(-0.02,-0.01)).mean("eV")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

cmap = plt.get_cmap('copper')
colors = [cmap(i) for i in np.linspace(0, 1, 5)]

binned = rebin(lt_an_p5, eV=3)
edc_an = binned.sel(eV=slice(-0.54,None), ky=slice(-0.6,-0.54), kx=slice(0.11,0.21)).sum("kx").sum("ky")
edc_n = binned.sel(eV=slice(-0.54,None), ky=slice(-0.32,-0.29), kx=slice(0.25,0.33)).sum("kx").sum("ky")
edc_an2 = binned.sel(eV=slice(-0.54,None), ky=slice(0.48,0.53), kx=slice(-0.23,-0.18)).sum("kx").sum("ky")
au_ref =rebin(corrected_edc,eV=3).sel(eV=slice(-0.24,None), phi=slice(-0.05,0.05)).sum("phi")

normalize(au_ref).plot(ax=ax, label='Gold Ref', marker='o', markeredgewidth=1, markerfacecolor='none', markeredgecolor=colors[1], color=colors[1])
normalize(edc_n).plot(ax=ax, label='Node', marker='o', markeredgewidth=1, markerfacecolor='none', markeredgecolor=colors[2], color=colors[2])
normalize(edc_an).plot(ax=ax, label='Antinode', marker='o', markeredgewidth=1, markerfacecolor='none', markeredgecolor=colors[3], color=colors[3])
# normalize(edc_an2).plot(ax=ax, label='AN')

ax.set_xlim(-0.24,0.1)
ax.set_ylim(-0.05,1.1)
ax.set_title('')
ax.set_ylabel('Intensity (arb. units)', fontsize=18, labelpad=10)
ax.set_xlabel('Binding Energy $E_B$ (eV)', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=16)

plt.legend(fontsize=12)
plt.savefig('Bi2223_EDCs.pdf', bbox_inches='tight')

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(5, 5))

cmap = plt.get_cmap('copper')
colors = [cmap(i) for i in np.linspace(0, 1, len(kxs)+3)]

i=0
for kx, ky in zip(kxs, kys):
    edc = lt_an_p5.sel(eV=slice(-0.1,0.1), kx=slice(kx-0.02,kx+0.02), ky=slice(ky-0.02,ky+0.02)).sum("kx").sum("ky")
    (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).plot(ax=ax, label=f'({kx}, {ky})', color=colors[i+1])
    ax.plot(-1*edc.sel(eV=slice(-0.08,0)).eV.values, (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).values, color=colors[i+1])
    i+=1

i=0
for kx, ky in zip(kxs, kys):
    edc = ht_an_p5.sel(eV=slice(-0.1,0.1), kx=slice(kx-0.02,kx+0.02), ky=slice(ky-0.02,ky+0.02)).sum("kx").sum("ky")
    (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).plot(ax=ax2, label=f'({kx}, {ky})', color=colors[i+1])
    ax2.plot(-1*edc.sel(eV=slice(-0.08,0)).eV.values, (normalize(edc)+i*0.8).sel(eV=slice(-0.08,0)).values, color=colors[i+1])
    i+=1

for axis in [ax, ax2]:
    axis.set_title('')
    axis.set_ylabel('', fontsize=18)
    axis.set_xlabel('', fontsize=18)
    axis.yaxis.set_ticks([])
    axis.yaxis.set_ticklabels([])
    axis.set_xlim(-0.08, 0.08)

ax.tick_params(axis='both', which='major', labelsize=16)
ax2.tick_params(axis='both', which='major', labelsize=16)
ax.set_ylabel('Intensity', fontsize=18)
ax.set_xlabel('E$_B$ (eV)', fontsize=18)
ax2.set_xlabel('E$_B$ (eV)', fontsize=18)

e=0.015
ax.axvline(x=-e, color='k', linestyle='--', linewidth=1)
ax.axvline(x=e, color='k', linestyle='--', linewidth=1)

plt.subplots_adjust(wspace=0.03)
# plt.savefig('Bi2223_gap_small.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 5))
lt_an_p5.sel(eV=slice(-0.04,0.02)).sum("eV").plot(ax=ax, cmap='PuBu_r', vmin=10, vmax=70, rasterized=True)

plt.ylim(-0.55, -0.1)
plt.xlim(0.3, 0.55)
plt.title('')
plt.ylabel('k$_y$ ($\AA^{-1}$)', fontsize=18)
plt.xlabel('k$_x$ ($\AA^{-1}$)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=16)


ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax, fraction=0.1, pad=0.05, aspect=25)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.ax.text(0.6, 1.0, 'Max', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.ax.text(0.6, -0.06, 'Min', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.set_ticks([])
# set x and y axis to be equal
# plt.gca().set_aspect('equal', adjustable='box')
# plt.savefig('Bi2223_FS_trilayer.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
lt_an_p5.sel(eV=slice(-0.01,0.01)).sum("eV").plot(ax=ax, cmap='PuBu_r', vmin=5, vmax=80, rasterized=True)

ax.set_ylim(-0.55, -0.)
ax.set_xlim(0.0, 0.55)
ax.set_title('')
ax.set_ylabel('k$_y$ ($\AA^{-1}$)', fontsize=18)
ax.set_xlabel('k$_x$ ($\AA^{-1}$)', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_xticks(np.arange(0, 0.55, 0.25))
ax.set_xticklabels(['0.0', '0.25', '0.5'])

ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax, fraction=0.1, pad=0.05, aspect=25)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.ax.text(0.6, 1.0, 'Max', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.ax.text(0.6, -0.06, 'Min', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.set_ticks([])
# set x and y axis to be equal
# plt.gca().set_aspect('equal', adjustable='box')
plt.savefig('Bi2223_FS_bilayer.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 5))

edm = lt_an_p5.sel(kx=slice(-0.4,0.01), eV=slice(-0.11,0.02), ky=slice(0.42,0.46)).sum("ky")
edm.plot(ax=ax, vmin=1, vmax=6, cmap='PuBu', rasterized=True)

ax.set_title('')
ax.set_xlim(-0.4,0.0)
ax.set_ylabel('Energy E$_B$ (eV)', fontsize=18)
ax.set_xlabel('k$_\parallel$ ($\AA^{-1}$)', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_xticks(np.arange(-0.4, 0.1, 0.2))
ax.collections[0].colorbar.remove()
cbar = plt.colorbar(ax.collections[0], ax=ax, fraction=0.1, pad=0.08, aspect=25)
cbar.set_label('Intensity', fontsize=16, labelpad=5)
cbar.ax.text(0.6, 1.0, 'Max', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.ax.text(0.6, -0.05, 'Min', ha='center', va='bottom', transform=cbar.ax.transAxes, fontsize=12)
cbar.set_ticks([])

# plt.savefig('Bi2223_EDM_splitting.pdf', bbox_inches='tight')
plt.show()

In [ ]:

fig, ax = plt.subplots(2, 5, figsize=(20, 8))

line = [-0.27, -0.25, -0.25, -0.25, -0.23, -0.23, -0.23, -0.23, -0.21, -0.21]

for i, ky in enumerate(lt_an_p5.sel(ky=slice(0.41,0.51)).ky):
    edm = lt_an_p5.sel(kx=slice(-0.55,0), eV=slice(-0.12,0.02), ky=ky)
    edm.plot(ax=ax.flat[i])
    ax.flat[i].set_title(f'ky={np.round(ky.values,2)}')
    ax.flat[i].axvline(x=line[i], color='k', linestyle='--', linewidth=1)

plt.tight_layout()

In [ ]:

fig, ax = plt.subplots(2, 5, figsize=(20, 8))

line = [-0.27, -0.25, -0.25, -0.25, -0.23, -0.23, -0.23, -0.23, -0.21, -0.21]

for i, ky in enumerate(ht_an_p5.sel(ky=slice(0.41,0.51)).ky):
    edm = ht_an_p5.sel(kx=slice(-0.55,0), eV=slice(-0.08,0.02), ky=ky)
    edm.plot(ax=ax.flat[i])
    ax.flat[i].set_title(f'ky={np.round(ky.values,2)}')
    ax.flat[i].axvline(x=line[i], color='k', linestyle='--', linewidth=1)

plt.tight_layout()

In [ ]:
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(5, 5))

line = [-0.27, -0.25, -0.25, -0.25, -0.23, -0.23, -0.23, -0.23, -0.21, -0.21]

for i, ky in enumerate(lt_an_p5.sel(ky=slice(0.41,0.51)).ky):
    edc = lt_an_p5.sel(kx=slice(line[i]-0.02,line[i]+0.02), eV=slice(-0.1,0.07), ky=ky).sum("kx")
    (normalize(edc)+i*1).sel(eV=slice(-0.08,0)).plot(ax=ax, label=f'kx={line[i]}, ky={np.round(ky,2)}', color=colors[i+1])
    ax.plot(-1*edc.sel(eV=slice(-0.08,0)).eV.values, (normalize(edc)+i*1).sel(eV=slice(-0.08,0)).values, color=colors[i+1])

for i, ky in enumerate(ht_an_p5.sel(ky=slice(0.41,0.51)).ky):
    edc = ht_an_p5.sel(kx=slice(line[i]-0.02,line[i]+0.02), eV=slice(-0.1,0.07), ky=ky).sum("kx")
    (normalize(edc)+i*1).sel(eV=slice(-0.08,0)).plot(ax=ax2, label=f'', color=colors[i+1])
    ax2.plot(-1*edc.sel(eV=slice(-0.08,0)).eV.values, (normalize(edc)+i*1).sel(eV=slice(-0.08,0)).values, color=colors[i+1])

for axis in [ax, ax2]:
    axis.set_title('')
    axis.set_ylabel('', fontsize=18)
    axis.set_xlabel('', fontsize=18)
    axis.yaxis.set_ticks([])
    axis.yaxis.set_ticklabels([])
    axis.set_xlim(-0.08, 0.08)

ax.tick_params(axis='both', which='major', labelsize=16)
ax2.tick_params(axis='both', which='major', labelsize=16)
ax.set_ylabel('Intensity', fontsize=18)
ax.set_xlabel('E$_B$ (eV)', fontsize=18)
ax2.set_xlabel('E$_B$ (eV)', fontsize=18)

e=0.03
ax.axvline(x=-e, color='k', linestyle='--', linewidth=1)
ax.axvline(x=e, color='k', linestyle='--', linewidth=1)

plt.subplots_adjust(wspace=0.03)
plt.savefig('Bi2223_gap_large.pdf', bbox_inches='tight')